> [!TIP]
> When?
> 
> get notified about any events that happen to the object they're observing.

Example:

* YouTube subscriptions: when a user subscribes to a channel, they get notified about new videos instead of having to check the channel manually.

The object that notifies other objects about the change to its state is called ***publisher***.

The objects that want to track changes to the publisher's state are called ***subscribers***.

In the following example, the Observer pattern let the text editor object notify other service objects about changes in its state.

![](../assets/observer-example.png)

In [1]:
from abc import ABC, abstractmethod


class EventManager:

    EVENT_OPEN = 1
    EVENT_SAVE = 2

    def __init__(self):
        self.listeners = {}

    def subscribe(self, event_type, listener):
        if event_type not in self.listeners:
            self.listeners[event_type] = []
        self.listeners[event_type].append(listener)

    def notify(self, event_type, event_data):
        if event_type in self.listeners:
            for listener in self.listeners[event_type]:
                listener.update(event_data)


class CEditor: # Controller, also the publisher

    def __init__(
            self, 
            event_manager: EventManager
    ):
        self.event_manager = event_manager

    def open_file(self, filepath):
        # do opening task here ...
        self.event_manager.notify(
            event_type=EventManager.EVENT_OPEN,
            event_data=filepath,
        )

    def save_file(self, filepath):
        # do saving task here
        self.event_manager.notify(
            event_type=EventManager.EVENT_SAVE,
            event_data=filepath,
        )


class VEditor:  # View

    def __init__(
            self,
            controller: CEditor,
    ):
        self.controller = controller

        while True:
            action = input("""What do you want to do: 
                1. open file
                2. save file
                q. Quit
                """
            )
            if action == "1":
                self.open_file()
            elif action == "2":
                self.save_file()
            elif action == "q":
                break

    def open_file(self):
        self.controller.open_file("path/to/file")
        
    def save_file(self):
        self.controller.save_file("new/path/to/file")


class EventListener(ABC):    # Interface

    @abstractmethod
    def update(self, event_data):
        pass


class EmailAlertsListener(EventListener): # implement interface, also a subscriber

    def update(self, filepath):
        print('[SENDING EMAIL]: Someone opened {}'.format(filepath))


if __name__ == "__main__":

    event_manager = EventManager()
    email_alert = EmailAlertsListener()

    event_manager.subscribe(
        event_type=EventManager.EVENT_OPEN,
        listener=email_alert
    )
    # can subsribe the email_alert to EVENT_SAVE as well

    controller = CEditor(event_manager)

    VEditor(controller)
    



[SENDING EMAIL]: Someone opened path/to/file
[SENDING EMAIL]: Someone opened path/to/file


In the above diagram and code, the editor class (i.e., the publisher) does not maintain the subscription list. Instead, it delegates this responsibility to a separate object called the `EventManager`.

You can upgrade the `EventManager` to serve as a centralized event dispatcher, letting any object act as a publisher. 

# References

1. https://refactoring.guru/design-patterns/observer